In [1]:
import math
import numpy as np
import pandas as pd
import catboost as cbt
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,precision_score, recall_score, f1_score 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold,StratifiedKFold,GridSearchCV
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
import gc
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
train = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\train_set\first_round_training_data - 副本.csv')
test = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\test_set\first_round_testing_data - 副本.csv')
submit = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\submit_example.csv')

In [3]:
quality_map = {'Excellent': 0, 'Good': 1, 'Pass': 2, 'Fail': 3}
train['label'] = train['Quality_label'].map(quality_map)
btype = 'cbt'
k = 5

In [4]:
parameter_name = ['Parameter{0}'.format(i) for i in range(5,11)]
attribute_name = ['Attribute{0}'.format(i) for i in range(4,9)]
X_train_para = train[parameter_name]
y_train_attr = train[attribute_name]
X_test_para = test[parameter_name]
y_train = train['label']
train_attr = np.zeros((X_train_para.shape[0],5))
test_attr = np.zeros((X_test_para.shape[0],5))
i = 0

for attr in attribute_name:
    kfold = KFold(n_splits=k, shuffle=True, random_state=2019)
    for train_index, test_index in kfold.split(X_train_para,y_train_attr):
        train_x, test_x, train_y, test_y = X_train_para.iloc[train_index], X_train_para.iloc[test_index], y_train_attr[attr].iloc[train_index], y_train_attr[attr].iloc[test_index]
        if btype == 'cbt':
            pipeline = Pipeline([('attr_model', CatBoostRegressor(random_state=2019))])
        elif btype == 'lgb':
            pipeline = Pipeline([('attr_model', LGBMRegressor(boosting_type="gbdt",random_state=2019))])
        elif btype == 'xgb':
            pipeline = Pipeline([('attr_model', XGBRegressor(random_state=2019))])

        n_estimators = range(500,1200,100)
        max_depth = [4,6,8,10]
        reg_alpha=[5,7,9,11,13]
        reg_lambda=[3,4,5,6]
        learning_rate=[0.01,0.03,0.05]
        random_grid = {
                        'attr_model__n_estimators': n_estimators,
                        'attr_model__max_depth': max_depth,
                        'attr_model__reg_alpha':reg_alpha,
                        'attr_model__reg_lambda':reg_lambda,
                        'attr_model__learning_rate':learning_rate
                       } 
        grid_search = GridSearchCV(pipeline,random_grid, n_jobs=-1, verbose=1, scoring=None)
        grid_search.fit(train_x,train_y)
        print('最佳效果：%0.3f' % grid_search.best_score_)
        best_parameters = grid_search.best_estimator_.get_params()
        print('最优参数：')
        for param_name in sorted(random_grid.keys()):
            print('\t%s: %r' % (param_name, best_parameters[param_name]))
        train_attr[test_index,i] += grid_search.predict(test_x)
        test_attr[:,i] += grid_search.predict(X_test_para)/k
    i += 1
print('mae',np.sum(np.absolute(train_attr - y_train_attr.values)/480))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


0:	learn: 287983.5718481	total: 67.7ms	remaining: 0us
最佳效果：184136.708
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


0:	learn: 287992.6323187	total: 17.9ms	remaining: 0us
最佳效果：183255.298
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

0:	learn: 31650405.2293204	total: 21.2ms	remaining: 0us
最佳效果：20693079.067
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits
0:	learn: 31589573.3010011	total: 27ms	remaining: 0us


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


最佳效果：20753789.941
最优参数：
	attr_model__n_estimators: 1
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


0:	learn: 4099413.9341210	total: 26.7ms	remaining: 0us
最佳效果：2711173.285
最优参数：
	attr_model__n_estimators: 1
mae 22362372.730338432


In [5]:
boosttype='cbt'
k=5

In [6]:
train[attribute_name] = train_attr
X_train = train[parameter_name+attribute_name]
y_train = train['label']
i = 0
for attr in attribute_name:
    test[attr] = test_attr[:,i]
    i += 1
X_test = test[parameter_name + attribute_name]
print(X_train.shape,X_test.shape)
oof = np.zeros((X_train.shape[0],4))
prediction = np.zeros((X_test.shape[0],4))
seeds = [x*x for x in range(10,20,2)]#设置随机种子
num_model_seed = len(seeds)
#params = {'max_depth':range(2, 7), 'n_estimators':range(100, 1100, 200), 'learning_rate':[0.01]}

for model_seed in range(num_model_seed):
    print(model_seed + 1)
    oof_model = np.zeros((X_train.shape[0],4))
    prediction_model=np.zeros((X_test.shape[0],4))
#    oof_xgb = np.zeros((X_train.shape[0],4))
#    prediction_xgb=np.zeros((X_test.shape[0],4))
    skf = StratifiedKFold(n_splits=k, random_state=seeds[model_seed], shuffle=True)#构造交叉验证集，进行按比例取样（splits表示测试集和验证集为4：1）
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print(index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y_train.iloc[train_index], y_train.iloc[test_index]
        gc.collect()#进行垃圾内存回收
        if boosttype=='cbt':
            model = cbt.CatBoostClassifier(iterations=1500,learning_rate=0.01,max_depth=6,verbose = 500,
                                           leaf_estimation_method='Newton',early_stopping_rounds=10000,
                                           task_type='GPU',loss_function='MultiClass')
            model.fit(train_x, train_y ,eval_set=(train_x, train_y))
#        xgb_model = GridSearchCV(XGBClassifier(),params,n_jobs=-1, cv=5, verbose=300,refit=True)
        elif boosttype=='xgb':
            model = XGBClassifier(
            silent=True ,
            learning_rate= 0.05, # 学习率
            min_child_weight=1,
            # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
            #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
            #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
            max_depth=7, # 构建树的深度，越大越容易过拟合
            gamma=0.1,  # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子。
            subsample=1, # 随机采样训练样本 训练实例的子采样比
            max_delta_step=0,#最大增量步长，我们允许每个树的权重估计。
            colsample_bytree=1, # 生成树时进行的列采样
            reg_lambda=1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
            #reg_alpha=0, # L1 正则项参数
            scale_pos_weight=1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重
            objective= 'multi:softmax', #多分类的问题 指定学习任务和相应的学习目标
            num_class=4, # 类别数，多分类与 multisoftmax 并用
            n_estimators=1200, #树的个数
            seed=2019, #随机种子
            eval_metric= 'mlogloss',
            )
            model.fit(train_x, train_y,eval_metric='mlogloss')
        elif boosttype=='lgb':
            model=LGBMClassifier(boosting_type="gbdt",num_leaves=23,reg_alpha=10, reg_lambda=5,max_depth=7,
                                 learning_rate=0.05, n_estimators=2000,colsample_bytree=0.7, subsample_freq=1,
                                 objective='multiclass', silent=True,subsample=0.7,min_child_samples=5,
                                 )
            model.fit(train_x, train_y, eval_set=(test_x, test_y),early_stopping_rounds=3000, verbose=1000,)#verbose=False   
        oof_model[test_index] = oof_model[test_index]+model.predict_proba(test_x) #预测验证集上某一个工件lebal为0到3的各自概率
        prediction_model = prediction_model+model.predict_proba(X_test)/k #预测测试集上某一个工件lebal为0到3的各自概率（除10是因为splits=10，故要进行10次轮换交叉验证）
    oof = oof+oof_model / num_model_seed
    prediction =prediction+prediction_model / num_model_seed

    print('logloss',log_loss(pd.get_dummies(y_train).values, oof_model))
    print('ac',accuracy_score(y_train, np.argmax(oof_model,axis=1)))
    print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof_model))/480))

print('logloss',log_loss(pd.get_dummies(y_train).values, oof))
print('ac',accuracy_score(y_train, np.argmax(oof,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y_train] - oof))/480))
beta=2
p1=precision_score(y_train,np.argmax(oof,axis=1),average='micro')
r1=recall_score(y_train,np.argmax(oof,axis=1),average='micro')
p2=precision_score(y_train,np.argmax(oof,axis=1),average='macro')
r2=recall_score(y_train,np.argmax(oof,axis=1),average='macro')
print('micro precision',p1)
print('micro recall',r1)
print('macro precision',p2)
print('macro recall',r2)
print('micro F2-Score',(1+beta*beta)*p1*r1/(beta*beta*p1+r1))
print('macro F2-Score',(1+beta*beta)*p2*r2/(beta*beta*p2+r2))

(6000, 11) (6000, 11)
1
0
0:	learn: -1.3812077	test: -1.3812077	best: -1.3812077 (0)	total: 42ms	remaining: 4.16s
99:	learn: -1.1654133	test: -1.1654133	best: -1.1654133 (99)	total: 1.48s	remaining: 0us
bestTest = -1.165413262
bestIteration = 99
Shrink model to first 100 iterations.
1
0:	learn: -1.3809371	test: -1.3809372	best: -1.3809372 (0)	total: 16ms	remaining: 1.59s
99:	learn: -1.1655067	test: -1.1655067	best: -1.1655067 (99)	total: 1.34s	remaining: 0us
bestTest = -1.165506685
bestIteration = 99
Shrink model to first 100 iterations.
2
0:	learn: -1.3808889	test: -1.3808890	best: -1.3808890 (0)	total: 14.3ms	remaining: 1.42s
99:	learn: -1.1636784	test: -1.1636784	best: -1.1636784 (99)	total: 1.34s	remaining: 0us
bestTest = -1.163678364
bestIteration = 99
Shrink model to first 100 iterations.
3
0:	learn: -1.3807654	test: -1.3807654	best: -1.3807654 (0)	total: 15.2ms	remaining: 1.51s
99:	learn: -1.1586535	test: -1.1586534	best: -1.1586534 (99)	total: 1.33s	remaining: 0us
bestTest = -1

In [7]:
sub = test[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]
for i, f in enumerate(prob_cols):
    sub[f] = prediction[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()

In [9]:
submit = sub.copy()
submit2 = sub.copy()
submit3 = sub.copy()
ratio = ['Excellent ratio','Good ratio','Pass ratio','Fail ratio']
submit[ratio]=submit[ratio]*50
submit2[ratio]=(submit2[ratio]*50).astype('int')
submit3[ratio]=(submit[ratio]-submit2[ratio]).astype('float')
submit2 = submit2.values
submit3 = submit3.values

In [10]:
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120): 
    if submit3[i,k[i]] > 0.5:
        submit2[i,k[i]] += 1
        submit3[i,k[i]] = 0
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120):
    if submit3[i,k[i]] > 0.5:
        submit2[i,k[i]] += 1
        submit3[i,k[i]] = 0
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120):    
    if sum(submit2[i,1:5]) < 50:
        if submit3[i,k[i]] > 0.5:
            submit2[i,k[i]] += 1
            submit3[i,k[i]] = 0
k = np.argmax(submit3[:,1:5],axis=1) + 1
for i in range(120):    
    if sum(submit2[i,1:5]) < 50:
        submit2[i,k[i]] += 1
        submit3[i,k[i]] = 0
submit[ratio] = submit2[:,1:5] / 50

In [11]:
sub.to_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\submit_example.csv',index=False)